## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from googleqa_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/95-googleq-a-train-bert-base-clean/model.h5'
BERT_PATH = '/kaggle/input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12'
TOKENIZER_PATH = BERT_PATH+'/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(TOKENIZER_PATH, True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

x = GlobalAveragePooling1D()(sequence_output)
x = Dropout(0.2)(x)
output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.942936,0.714651,0.497524,0.501214,0.395950,0.510998,0.613335,0.635701,0.496878,...,0.933856,0.893332,0.565106,0.977139,0.949980,0.850498,0.026054,0.029164,0.846790,0.922801
1,46,0.935254,0.550237,0.007867,0.679704,0.793657,0.907614,0.511677,0.499498,0.157369,...,0.772236,0.950268,0.631725,0.973895,0.981824,0.859358,0.972946,0.068310,0.044533,0.881697
2,70,0.924789,0.729784,0.040735,0.705179,0.913858,0.943275,0.495098,0.583202,0.139401,...,0.867026,0.893841,0.646379,0.975516,0.950028,0.792719,0.039594,0.067606,0.849879,0.905884
3,132,0.894291,0.466489,0.009602,0.655007,0.789198,0.917664,0.575006,0.549747,0.079977,...,0.761225,0.920910,0.655553,0.972324,0.981680,0.870975,0.833590,0.149592,0.389317,0.895996
4,200,0.937254,0.420934,0.045931,0.828164,0.755234,0.861752,0.605238,0.679797,0.027501,...,0.757915,0.901625,0.649068,0.981254,0.971816,0.826643,0.107135,0.094449,0.497951,0.925909


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.911180,0.624239,0.046247,0.685697,0.806416,0.846291,0.574931,0.526467,0.226001,...,0.840328,0.910241,0.664492,0.965822,0.969557,0.845709,0.519421,0.134660,0.487288,0.912045
std,2812.670060,0.042928,0.133113,0.075858,0.120027,0.106890,0.137792,0.049453,0.087177,0.180926,...,0.072249,0.034661,0.063453,0.013060,0.014445,0.050904,0.347464,0.072645,0.286954,0.024553
min,39.000000,0.736038,0.381090,0.004255,0.187871,0.282459,0.175075,0.434534,0.349649,0.005128,...,0.644538,0.737580,0.493064,0.902152,0.902717,0.644520,0.004343,0.004865,0.007098,0.826882
25%,2572.000000,0.887199,0.507388,0.010709,0.607571,0.768548,0.827058,0.543015,0.468169,0.088207,...,0.780303,0.893252,0.620597,0.958499,0.961488,0.812310,0.147963,0.080253,0.229399,0.897085
50%,5093.000000,0.918276,0.603246,0.016326,0.678785,0.820884,0.897032,0.571700,0.502774,0.167353,...,0.846549,0.912221,0.659294,0.967413,0.972603,0.851900,0.594405,0.128962,0.502358,0.913332
75%,7482.000000,0.943967,0.740914,0.040428,0.770994,0.868719,0.930334,0.603617,0.578162,0.321867,...,0.901382,0.934118,0.705772,0.974760,0.979892,0.882323,0.850956,0.181704,0.713484,0.930564
max,9640.000000,0.981502,0.920177,0.551683,0.983174,0.978403,0.988335,0.715081,0.822762,0.770163,...,0.968377,0.975950,0.845149,0.992201,0.991897,0.955721,0.986571,0.385089,0.987875,0.960311
